In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import myslack
import datetime as dt
import pickle
import time
import re
from tqdm import trange

In [2]:
driver = webdriver.Chrome()  
url = 'http://produce48.mnet.com/pc/profile/0'
driver.get(url) 

In [84]:
review_count = int(driver.find_element_by_xpath('//*[@id="comment"]/div/h2/strong').text)
comment_page_count = int(review_count/20)+1
review_count, comment_page_count

(4362, 219)

In [85]:
def open_all_comment():
    print('이전 댓글 모두 불러오기 시작...')    
    for _ in trange(comment_page_count):
        try:
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/a').click()
            time.sleep(0.3)
        except Exception as e:
            print(e)
            time.sleep(1)
            pass
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    print('이전 댓글 모두 불러오기 성공!')

In [86]:
def make_produce48_df(trainee_name):
    print(trainee_name+'의 댓글 수: '+str(review_count))
    open_all_comment()
    
    print('making dataframe...')
    df = pd.DataFrame(columns=['User_ID','Comment','Date'])
    for i in trange(review_count):
        try:
            df.loc[len(df)] = { 
            'User_ID':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/strong'.format(i+1)).text,
            'Comment':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/p'.format(i+1)).text,
            'Date':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/span'.format(i+1)).text.split(' ')[0],
            }
        except Exception as e:
            print(e)
            pass
    print('making dataframe done!')
    return df

In [87]:
def make_df_name():
    eng_name = \
    re.sub("[^a-zA-Z]"," ",driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[1]/dl[2]/dt/p[2]').text).strip()
    result = eng_name.replace(' ','_')
    return result

name = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[1]/dl[2]/dt/p[2]/span').text
agency = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/h4/span').text

make_df_name(), name, agency

('MURASE_SAE', '무라세 사에', 'NMB48')

In [88]:
%time MURASE_SAE = make_produce48_df(name)

  0%|          | 0/219 [00:00<?, ?it/s]

무라세 사에의 댓글 수: 4362
이전 댓글 모두 불러오기 시작...


100%|█████████▉| 218/219 [16:34<00:04,  4.56s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="comment"]/div/div[2]/div/a"}
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.13.6 x86_64)



  0%|          | 0/4362 [00:00<?, ?it/s]

이전 댓글 모두 불러오기 성공!
making dataframe...


100%|██████████| 4362/4362 [48:27<00:00,  1.50it/s] 

making dataframe done!
CPU times: user 37.2 s, sys: 3.22 s, total: 40.4 s
Wall time: 1h 5min 2s


In [89]:
MURASE_SAE.shape

(4362, 3)

In [90]:
MURASE_SAE.tail()

,User_ID,Comment,Date
4357,정동욱,뽑고싶은분이 너무 많네요ㅎㅎ 예쁘십니다!,2018-05-13
4358,김윤호,사에삐는 워낙 이쁘니까,2018-05-13
4359,메리,사에삐 귀여워요ㅠㅠㅠㅠ소중한 한표 많이많이 투표합시다 ㅜㅜㅜㅜ 사에 데뷔하자ㅠㅠㅠㅠ...,2018-05-13
4360,Kyoungsurk Kam,앞으로 기대할께요...열심히 해주시기를^^,2018-05-13
4361,이유림,사에 너무 예쁘고 귀엽지 않나요 여러분의 한표를 던져주세요 너무 센스있고 귀여운 아...,2018-05-13


In [91]:
pickle.dump(MURASE_SAE, open('./produce48/NMB48/MURASE_SAE.pkl','wb'))

In [92]:
MURASE_SAE_df = pickle.load(open('./produce48/NMB48/MURASE_SAE.pkl','rb'))

In [93]:
myslack.send_slack('{} finish!'.format(make_df_name()))

<Response [200]>


In [94]:
MURASE_SAE_df.tail()

,User_ID,Comment,Date
4357,정동욱,뽑고싶은분이 너무 많네요ㅎㅎ 예쁘십니다!,2018-05-13
4358,김윤호,사에삐는 워낙 이쁘니까,2018-05-13
4359,메리,사에삐 귀여워요ㅠㅠㅠㅠ소중한 한표 많이많이 투표합시다 ㅜㅜㅜㅜ 사에 데뷔하자ㅠㅠㅠㅠ...,2018-05-13
4360,Kyoungsurk Kam,앞으로 기대할께요...열심히 해주시기를^^,2018-05-13
4361,이유림,사에 너무 예쁘고 귀엽지 않나요 여러분의 한표를 던져주세요 너무 센스있고 귀여운 아...,2018-05-13


In [95]:
driver.close() # 창닫기
driver.quit() # 브라우져 닫기